# 简单项目2：回归模型评估系统

## 学习目标
- 理解回归模型的评估指标（MSE、RMSE、MAE、R²）
- 使用scikit-learn计算回归评估指标
- 进行回归模型的交叉验证
- 可视化回归模型的预测结果
- 进行残差分析

## 项目概述
本项目将构建一个完整的回归模型评估系统，包括：
1. 数据准备和预处理
2. 训练多个回归模型
3. 计算回归评估指标
4. 交叉验证
5. 可视化评估结果和残差分析


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_boston, make_regression

# 设置中文字体（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 设置随机种子，确保结果可复现
np.random.seed(42)

print("环境准备完成！")


## 1. 数据准备

我们将使用scikit-learn内置的回归数据集，或者生成模拟数据。


In [ ]:
# 生成回归数据集
# 使用make_regression生成模拟数据，适合学习和演示
X, y = make_regression(n_samples=500, n_features=10, n_informative=5, 
                       noise=20, random_state=42)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("=" * 60)
print("数据信息")
print("=" * 60)
print(f"训练集样本数: {len(X_train)}")
print(f"测试集样本数: {len(X_test)}")
print(f"特征数: {X.shape[1]}")
print(f"目标值范围: [{y.min():.2f}, {y.max():.2f}]")
print(f"目标值均值: {y.mean():.2f}")
print(f"目标值标准差: {y.std():.2f}")


## 2. 训练多个回归模型

我们将训练三个不同的回归模型：
- 线性回归：简单快速的基准模型
- 随机森林回归：集成学习方法
- 梯度提升回归：强大的集成学习方法


In [ ]:
# 创建模型字典
models = {
    '线性回归': LinearRegression(),
    '随机森林': RandomForestRegressor(n_estimators=100, random_state=42),
    '梯度提升': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# 训练模型
trained_models = {}
print("=" * 60)
print("训练模型")
print("=" * 60)

for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model
    print(f"✓ {name} 训练完成")


## 3. 计算回归评估指标

回归模型的评估指标包括：
- **MSE（均方误差）**：预测值与真实值差的平方的平均值
- **RMSE（均方根误差）**：MSE的平方根，与目标值单位相同
- **MAE（平均绝对误差）**：预测值与真实值差的绝对值的平均值
- **R²（决定系数）**：模型解释的方差比例，越接近1越好


In [ ]:
# 评估每个模型
results = {}

print("=" * 60)
print("模型评估结果")
print("=" * 60)

for name, model in trained_models.items():
    # 进行预测
    y_pred = model.predict(X_test)
    
    # 计算评估指标
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # 保存结果
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'y_pred': y_pred
    }
    
    # 打印结果
    print(f"\n{name}:")
    print(f"  MSE:  {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")

# 创建评估指标对比表
print("\n" + "=" * 60)
print("评估指标对比表")
print("=" * 60)
df_results = pd.DataFrame({
    name: {
        'MSE': f"{results[name]['MSE']:.4f}",
        'RMSE': f"{results[name]['RMSE']:.4f}",
        'MAE': f"{results[name]['MAE']:.4f}",
        'R²': f"{results[name]['R²']:.4f}"
    }
    for name in results.keys()
}).T
print(df_results)


In [ ]:
# 创建可视化图表
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 左上：评估指标对比
ax1 = axes[0, 0]
metrics = ['MSE', 'RMSE', 'MAE', 'R²']
x = np.arange(len(metrics))
width = 0.25

for i, (name, result) in enumerate(results.items()):
    # 注意：R²需要单独处理，因为它的范围不同
    values = [result['MSE']/1000, result['RMSE']/10, result['MAE']/10, result['R²']]
    ax1.bar(x + i * width, values, width, label=name, alpha=0.7)

ax1.set_xlabel('评估指标', fontsize=12)
ax1.set_ylabel('标准化分数', fontsize=12)
ax1.set_title('模型评估指标对比（标准化）', fontsize=14)
ax1.set_xticks(x + width)
ax1.set_xticklabels(metrics)
ax1.legend()
ax1.grid(True, axis='y', alpha=0.3)

# 右上：真实值 vs 预测值（线性回归）
ax2 = axes[0, 1]
name = '线性回归'
y_pred = results[name]['y_pred']
ax2.scatter(y_test, y_pred, alpha=0.6, edgecolors='black', linewidth=0.5)
# 添加理想预测线（y=x）
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
ax2.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='理想预测线')
ax2.set_xlabel('真实值', fontsize=12)
ax2.set_ylabel('预测值', fontsize=12)
ax2.set_title(f'{name} - 真实值 vs 预测值', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 左下：残差分布图（线性回归）
ax3 = axes[1, 0]
residuals = y_test - y_pred
ax3.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
ax3.axvline(x=0, color='r', linestyle='--', linewidth=2, label='零残差线')
ax3.set_xlabel('残差', fontsize=12)
ax3.set_ylabel('频数', fontsize=12)
ax3.set_title(f'{name} - 残差分布', fontsize=14)
ax3.legend()
ax3.grid(True, axis='y', alpha=0.3)

# 右下：残差 vs 预测值（线性回归）
ax4 = axes[1, 1]
ax4.scatter(y_pred, residuals, alpha=0.6, edgecolors='black', linewidth=0.5)
ax4.axhline(y=0, color='r', linestyle='--', linewidth=2, label='零残差线')
ax4.set_xlabel('预测值', fontsize=12)
ax4.set_ylabel('残差', fontsize=12)
ax4.set_title(f'{name} - 残差 vs 预测值', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('regression_evaluation_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 使用5折交叉验证
cv = KFold(n_splits=5, shuffle=True, random_state=42)

print("=" * 60)
print("交叉验证结果")
print("=" * 60)

cv_results = {}
for name, model in models.items():
    # 使用负均方误差作为评分（scikit-learn默认使用负分数，越大越好）
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    # 转换为RMSE
    rmse_scores = np.sqrt(-scores)
    cv_results[name] = rmse_scores
    
    print(f"\n{name}:")
    print(f"  RMSE: {rmse_scores.mean():.4f} ± {rmse_scores.std():.4f}")
    print(f"  各折得分: {rmse_scores}")

# 可视化交叉验证结果
plt.figure(figsize=(10, 6))
positions = np.arange(len(models))
means = [cv_results[name].mean() for name in models.keys()]
stds = [cv_results[name].std() for name in models.keys()]

plt.bar(positions, means, yerr=stds, alpha=0.7, 
        color=['skyblue', 'lightgreen', 'lightcoral'],
        edgecolor='black', capsize=5)
plt.xticks(positions, models.keys(), fontsize=12)
plt.ylabel('RMSE', fontsize=12)
plt.title('交叉验证结果（5折）- RMSE', fontsize=14)
plt.grid(True, axis='y', alpha=0.3)

# 添加数值标签
for i, (mean, std) in enumerate(zip(means, stds)):
    plt.text(i, mean + std + 0.5, f'{mean:.2f}±{std:.2f}',
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('cross_validation_results.png', dpi=300, bbox_inches='tight')
plt.show()


## 6. 总结与思考

### 关键知识点总结

1. **回归评估指标**：
   - MSE：对异常值敏感
   - RMSE：与目标值单位相同，易于理解
   - MAE：对异常值不敏感
   - R²：衡量模型解释的方差比例

2. **残差分析**：
   - 残差应该随机分布，不应该有模式
   - 残差分布应该接近正态分布
   - 残差 vs 预测值图应该没有明显的趋势

3. **交叉验证**：
   - 可以更准确地评估模型性能
   - 减少数据划分的随机性影响
   - 提供性能的置信区间

### 思考问题

1. 为什么R²分数接近1不一定意味着模型很好？
2. 如何根据残差图判断模型是否需要改进？
3. 什么时候应该使用MAE而不是RMSE？
4. 交叉验证的折数应该如何选择？

### 下一步学习

- 尝试不同的回归模型
- 添加特征工程步骤
- 实现自定义评估指标
- 学习模型解释性分析（SHAP值等）
